In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch("http://localhost:9200")

In [64]:
query = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}

index_name = "idx_ifab_docs"

fields=["id","h1", "h2", "h3", "text" , "url"]

response = es.search(index=index_name, body=query, source=fields)

/tmp/ipykernel_8261/2977911075.py:15: DeprecationWarning: Received 'source' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=query, source=fields)


In [65]:
context_docs = []

for r in response["hits"]["hits"]:
    headers = [ h.replace('\n',' ') for h in [ r['_source']['h1'], r['_source']['h2'], r['_source']['h3'] ]  ]
    topic_name = '. '.join(headers)
    context_docs.append(
        {
         'id':r['_source']['id'],
         'topic_name':topic_name,
         'url':r['_source']['url'],
         'topic_text':{r['_source']['text']}         
        }
    )    

In [47]:
print(context_docs)

[{'id': '3be61ccf4553d4b8120dc4b8810de722', 'topic_name': 'ABOUT THE LAWS. Philosophy and Spirit. ', 'url': 'https://www.theifab.com/laws/latest/about-the-laws/', 'topic_text': {'Football is the greatest sport on earth. It is played on every continent, in every country and at many different levels. The fact that the Laws of the Game are the same for all football throughout the world, from the FIFA World Cup™ through to a game between young children in a remote village, is a considerable strength which must continue to be harnessed for the good of football everywhere.\nFootball must have Laws which keep the game fair – this is a crucial foundation of the ‘beautiful game’ and a vital feature of the ‘spirit’ of the game. The best matches are those where the referee is rarely needed because the players play with respect for each other, the match officials and the Laws.\nFootball’s Laws are relatively simple compared to most other team sports, but as many situations are subjective and match

In [66]:
def get_prompt(c):
    context_string = f"""        
    Topic: {c['topic_name']}
    Topic content:
        {c['topic_text']}
    \n
    """
    prompt = f"""
    You are an expert in footbal rules. Prepare 5 questions for students based on provided Topic and it's content.    
    
    {context_string}    
    
    Provide the output in parsable JSON without using code blocks:
    ["question1", "question2", ..., "question5"] 
    """.strip()
    
    return(prompt)

In [61]:
from openai import OpenAI
client = OpenAI()

In [68]:
# Generate 5 questions for each topic using gpt-4o

# generated_qusetions = []
# for c in context_docs:
#     # print(get_prompt(c))
#     prompt = get_prompt(c) 

#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )

#     json_response = response.choices[0].message.content

#     c['questions'] = json_responseselecыду
    
#     generated_qusetions.append(c)  

      
  

In [4]:
import pandas as pd

In [3]:
# save generated questions to parquet 


# df = pd.DataFrame(generated_qusetions)
# df.to_parquet('generated_questions.parquet')

In [10]:
import json 

In [32]:
docs = pd.read_parquet('generated_questions.parquet').to_dict(orient='records')

In [53]:
from sentence_transformers import SentenceTransformer
model_name = 'all-mpnet-base-v2'
model = SentenceTransformer(model_name)

/home/azureuser/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/azureuser/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
# generate ground-truth dataset which  contain question and a topic

error_lst = []
qna_collection = []       
    
for d in docs:    
    q_str = d['questions'].replace('\n','')    
    try:
        questions = json.loads(q_str)        
        for q in questions:
            embedding = model.encode(q)
            qna_collection.append({
             'doc_id':d['id'],
             'topic_name':d['topic_name'],
             'url':d['url'],
             'topic_text':d['topic_text'],
             'question':q,
             'embdding':embedding
            })
    except:
        error_lst.append(t)
    

    
    

In [57]:
df = pd.DataFrame(qna_collection)
df.to_parquet('generated_questions_embedded.parquet')

In [5]:
ground_truth_docs = pd.read_parquet('generated_questions_embedded.parquet').to_dict(orient='records')

In [6]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [7]:
#  calculate matrix results for vector search

relevance_total_vector =[]

for gr in ground_truth_docs:
    q = gr['question']
    vector = gr['embdding']
    id_correct = gr['doc_id']
    # print(id_correct,q)
    
    index_name = "idx_ifab_docs"
    
    query = {
        "field": "embedding",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000, 
    }
    
    res = es.search(index=index_name, knn=query, source=["id", "text"])
    
    relevance = []
    
    for r in res["hits"]["hits"]:    
        relevance.append(r['_source']['id'] == id_correct)
    
    relevance_total_vector.append(relevance)
    

In [8]:

#  calculate matrix results for text search
relevance_total_text =[]

for gr in ground_truth_docs:
    q = gr['question']
    vector = gr['embdding']
    id_correct = gr['doc_id']
    # print(id_correct,q)
    
    index_name = "idx_ifab_docs"
    
    # query = {
    #     "field": "embedding",
    #     "query_vector": vector,
    #     "k": 5,
    #     "num_candidates": 10000, 
    # }

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": q,                        
                        "fields": ["text"],
                        "type": "best_fields"
                    }
                }                
            }
        }
    }



    response = es.search(index=index_name, body=search_query)
    
    # res = es.search(index=index_name, knn=query, source=["id", "text"])
    
    relevance = []
    
    for r in res["hits"]["hits"]:    
        relevance.append(r['_source']['id'] == id_correct)
    
    relevance_total_text.append(relevance)
    

In [9]:
print(f'Vector-search hit-rate:{hit_rate(relevance_total_vector)}')
print(f'Vector-search MRR:{mrr(relevance_total_vector)}')

Vector-search hit-rate:0.810394060536836
Vector-search MRR:0.653131543879688


In [10]:
print(f'Text-search hit-rate:{hit_rate(relevance_total_text)}')
print(f'Text-search MRR:{mrr(relevance_total_text)}')

Text-search hit-rate:0.014277555682467162
Text-search MRR:0.0065200837616600045
